In [3]:
import pyagrum.lib.notebook as gnb
import pyagrum as gum

In [14]:
bn = gum.BayesNet("IncidentResponseNetwork")

RE = bn.add(gum.LabelizedVariable('RE', 'Recent Deployment', 2))
HE = bn.add(gum.LabelizedVariable('HE', 'High Error Rate', 2))
SL = bn.add(gum.LabelizedVariable('SL', 'Slowdown', 2))
LC = bn.add(gum.LabelizedVariable('LC', 'High Load on our Services', 2))
QQ = bn.add(gum.LabelizedVariable('QQ', 'Dangerous Queue Status', 2))
DB = bn.add(gum.LabelizedVariable('DB', 'Dangerous Database Status', 2))
FE = bn.add(gum.LabelizedVariable('FE', 'Failing External Service', 2))
IS = bn.add(gum.LabelizedVariable('IS', 'Infrastructure Scaling Failure', 2))
UU = bn.add(gum.LabelizedVariable('UU', 'High User Load', 2))
AL = bn.add(gum.LabelizedVariable('AL', 'Alerts Triggered', 2))
CR = bn.add(gum.LabelizedVariable('CR', 'Severity', 3))
PR = bn.add(gum.LabelizedVariable('PR', 'Post-Incident Response published', 2))

bn.addArc("RE", "HE")
bn.addArc("RE", "SL")
bn.addArc("UU", "LC")
bn.addArc("IS", "LC")
bn.addArc("IS", "SL")
bn.addArc("LC", "QQ")
bn.addArc("LC", "DB")
bn.addArc("DB", "SL")
bn.addArc("DB", "HE")
bn.addArc("DB", "AL")
bn.addArc("FE", "HE")
bn.addArc("FE", "SL")
bn.addArc("QQ", "SL")
bn.addArc("QQ", "AL")
bn.addArc("HE", "AL")
bn.addArc("HE", "CR")
bn.addArc("SL", "AL")
bn.addArc("SL", "CR")
bn.addArc("CR", "PR")
bn.addArc("AL", "PR")

print(bn)
bn

BN{nodes: 12, arcs: 20, domainSize: 6144, dim: 82, mem: 1Ko 256o}


(pyagrum.BayesNet@0x308ca380) BN{nodes: 12, arcs: 20, domainSize: 6144, dim: 82, mem: 1Ko 256o}

In [53]:
expert_data = {
    # Root
    "P_UU": 0.5,
    "P_IS": 0.1,
    "P_FE": 0.2,
    "P_RE": 0.9,
    
    # High Service Load
    "P_UU_LC": 0.8,
    "P_IS_LC": 0.7,
    "P_UU_AND_IS_LC": 0.9,
    "P_NOT_UU_NOT_IS_LC": 0.1,
    
    # Dangerous Queue
    "P_LC_QQ": 0.75,
    "P_NOT_LC_QQ": 0.2,
    
    # Dangerous Database
    "P_LC_DB": 0.65,
    "P_NOT_LC_DB":0.2,
    
    # Error Rate
    "P_RE_HE": 0.6,   
    "P_FE_HE": 0.7,   
    "P_DB_HE": 0.5,   
    
    # Slowdown
    "P_QQ_SL": 0.8,   
    "P_DB_SL": 0.7,   
    "P_IS_SL": 0.6,   
    "P_RE_SL": 0.3,   
    "P_FE_SL": 0.4,   
    
    # Alerts
    "P_QQ_AL": 0.85,  
    "P_DB_AL": 0.8,   
    "P_SL_AL": 0.75,  
    "P_HE_AL": 0.9,   
    
    # Severity
    "P_SL_CR_P0": 0.4,  
    "P_SL_CR_P1": 0.5,  
    "P_HE_CR_P0": 0.5,  
    "P_HE_CR_P1": 0.45, 
    "P_NOT_SL_NOT_HE_CR_P0": 0.1,  
    "P_NOT_SL_NOT_HE_CR_P1": 0.2,  
    "P_SL_AND_HE_CR_P0": 0.8,  
    "P_SL_AND_HE_CR_P1": 0.15, 
    
    # Post-Incident Response
    "P_CR_P0_NOT_AL_PR": 0.3,  
    "P_CR_P1_NOT_AL_PR": 0.2,  
    "P_CR_P2_NOT_AL_PR": 0.1,  
    "P_CR_P0_AND_AL_PR": 0.95, 
    "P_CR_P1_AND_AL_PR": 0.8,  
    "P_CR_P2_AND_AL_PR": 0.5   
}
print(len(expert_data))

38


In [54]:
def create_cpt(node, idd):
    bn.cpt(UU).fillWith([1-expert_data[idd],expert_data[idd]])
    print(bn.cpt(UU))

create_cpt(UU, "P_UU")
create_cpt(IS, "P_IS")
create_cpt(FE, "P_FE")
create_cpt(RE, "P_RE")

bn.cpt(LC).fillWith([
    1-expert_data["P_NOT_UU_NOT_IS_LC"],expert_data["P_NOT_UU_NOT_IS_LC"],
    1-expert_data["P_UU_LC"],expert_data["P_UU_LC"],
    1-expert_data["P_IS_LC"],expert_data["P_IS_LC"],
    1-expert_data["P_UU_AND_IS_LC"],expert_data["P_UU_AND_IS_LC"],
])
bn.cpt(LC)

bn.cpt(QQ).fillWith([
    1-expert_data["P_NOT_LC_QQ"], expert_data["P_NOT_LC_QQ"],
    1-expert_data["P_LC_QQ"],expert_data["P_LC_QQ"]
])
bn.cpt(QQ)


  UU               |
0        |1        |
---------|---------|
 0.5000  | 0.5000  |


  UU               |
0        |1        |
---------|---------|
 0.9000  | 0.1000  |


  UU               |
0        |1        |
---------|---------|
 0.8000  | 0.2000  |


  UU               |
0        |1        |
---------|---------|
 0.1000  | 0.9000  |



(pyagrum.Tensor@0x2f5f0ea0) 
      ||  QQ               |
LC    ||0        |1        |
------||---------|---------|
0     || 0.8000  | 0.2000  |
1     || 0.2500  | 0.7500  |